In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier

from utility import make_BMI

In [2]:
data = pd.read_csv('../data/train.csv', index_col='id')

data.head()

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
id,,,,,,,,,,,,,,,,,
0,Male,24.443011,1.699998,81.669950,yes,yes,2.000000,2.983297,Sometimes,no,2.763573,no,0.000000,0.976473,Sometimes,Public_Transportation,Overweight_Level_II
1,Female,18.000000,1.560000,57.000000,yes,yes,2.000000,3.000000,Frequently,no,2.000000,no,1.000000,1.000000,no,Automobile,Normal_Weight
2,Female,18.000000,1.711460,50.165754,yes,yes,1.880534,1.411685,Sometimes,no,1.910378,no,0.866045,1.673584,no,Public_Transportation,Insufficient_Weight
3,Female,20.952737,1.710730,131.274851,yes,yes,3.000000,3.000000,Sometimes,no,1.674061,no,1.467863,0.780199,Sometimes,Public_Transportation,Obesity_Type_III
4,Male,31.641081,1.914186,93.798055,yes,yes,2.679664,1.971472,Sometimes,no,1.979848,no,1.967973,0.931721,Sometimes,Public_Transportation,Overweight_Level_II


На основании проведенного EDA, мы знаем, что наиболее информативными являются признаки веса и роста, кроме того они коррелируют, поэтому, сделаем из них одну - ИМТ.
Остальные колонки оставим без изменений и построим модель классификации на catboost, это будет нашей отправной точкой.
Градиентный бустинг выбран в связи с наличием выбросов в данных, а также в связи с тем, что по некоторым категориальным признакам часть данных хорошо разделима (смотри EDA).

In [3]:
data['BMI'] = make_BMI(data)
data = data.drop(['Height', 'Weight'], axis=1)
data = pd.get_dummies(data, columns=['Gender', 
                                     'family_history_with_overweight',
                                     'FAVC',
                                     'SMOKE',
                                     'SCC'], drop_first=True)
cat = ['CAEC', 'CALC', 'MTRANS']

In [4]:
X_train, X_valid, y_train, y_valid = train_test_split(data.drop(['NObeyesdad'], axis=1),
                                                      data['NObeyesdad'],
                                                      test_size=0.20,
                                                      random_state=3,
                                                      shuffle=True)

In [5]:
clf = CatBoostClassifier()
clf.fit(X_train, y_train, eval_set=(X_valid, y_valid), cat_features=cat, plot=True, silent=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

#### График обучения
![График обучения](https://s999sas.storage.yandex.net/rdisk/98c98963b4eddfc323923cb03a3e2a3c81d4e859aa1d84ac13051c38d435632c/65e4be7e/LyPBJX7DrmJ2vc9Oq453nfRYDnCUQzot78T_e-65evW9yZtpLl0NTCkdaiFdQRN2En4-ctQf3e4KKWwBCDviQw==?uid=285904750&filename=base.png&disposition=inline&hash=&limit=0&content_type=image%2Fpng&owner_uid=285904750&fsize=21473&hid=fdf8bbe43ea6956d6f99c4325fd6d5c1&media_type=image&tknv=v2&etag=be511c7bfa2a6d2e1c2dd13bc495a797&rtoken=aSkwurm5cXGX&force_default=yes&ycrid=na-0dce424b2410b0caf1e0f797fec9fa47-downloader10f&ts=612c59bae1b80&s=41137ee84c1933d7c231e23a984cef6213e90cdefa8971d6abe16dc453805f32&pb=U2FsdGVkX1_ScQlSIpwbFElrqxomlJWIvSTw0GMRuVmCs2ZC3J8qVnEhrGPlzXNqFw4eTO6LJzgzrRAadYK1uTdwYkbXtdi-WiBgT-O8QkU)

In [6]:
clf.score(X_train, y_train)

0.9083463808262074

In [7]:
clf.score(X_valid, y_valid)

0.8853564547206165

0.884 - базовая accuracy от которого будем отталкиваться

*Так как я изначально не зафиксировал random_state у классификатора, при повторном прогоне точность оказалась выше*

Подготавливаю данные для отправки, чтобы оценить accuracy базовой модели на public.

In [8]:
test = pd.read_csv('../data/test.csv', index_col='id')

test['BMI'] = make_BMI(test)
test = test.drop(['Height', 'Weight'], axis=1)
test = pd.get_dummies(test, columns=['Gender', 
                                     'family_history_with_overweight',
                                     'FAVC',
                                     'SMOKE',
                                     'SCC'], drop_first=True)
pred = clf.predict(test)
pred

array([['Obesity_Type_II'],
       ['Overweight_Level_I'],
       ['Obesity_Type_III'],
       ...,
       ['Insufficient_Weight'],
       ['Normal_Weight'],
       ['Obesity_Type_II']], dtype=object)

In [10]:
pd.Series(pred[:, 0], index=test.index).to_csv('../preds/pred0.csv')

Piblic score: 0.88186